In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import enerflow as ef

from Utilities.PV_AC_Output import PV_AC_Output
from Utilities.Data_import import calculate_retail_price
from Utilities import Plot, KPI

from Models.PV_BESS_LP_opt import PV_BESS_LP_solver
from Models.PV_BESS_LP_time_horizon import PV_BESS_LP_Time_horizon_v2

In [25]:
df = pd.read_csv("data/electricity_demand.csv", index_col=0)
file_path = "data/meteo_data.csv"
df_meteo = pd.read_csv("data/meteo_data.csv", index_col=0)


In [26]:
df_meteo

,AirPressure,Kvalitet [AirPressure],AirTemperature,Kvalitet [AirTemperature],DewPointTemperature,Kvalitet [DewPointTemperature],RelativeHumidity,Kvalitet [RelativeHumidity],SolarGHI,Kvalitet [SolarGHI],WindSpeed,Kvalitet [WindSpeed],WindDirection,Kvalitet [WindDirection]
2019-01-01 00:00:00,996.2,G,5.7,G,4.6,G,93.0,G,0.0,G,7.0,G,210.0,G
2019-01-01 01:00:00,994.4,G,5.9,G,4.5,G,91.0,G,0.0,G,6.0,G,230.0,G
2019-01-01 02:00:00,992.8,G,6.2,G,3.5,G,83.0,G,0.0,G,8.0,G,250.0,G
2019-01-01 03:00:00,991.4,G,5.3,G,1.1,G,74.0,G,0.0,G,7.0,G,250.0,G
2019-01-01 04:00:00,990.3,G,5.1,G,0.5,G,72.0,G,0.0,G,9.0,G,240.0,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,1023.3,G,1.1,G,-4.6,G,66.0,Y,0.0,G,2.0,G,130.0,G
2019-12-31 20:00:00,1022.8,G,1.6,G,-4.3,G,65.0,Y,0.0,G,4.0,G,250.0,G
2019-12-31 21:00:00,1022.6,G,1.7,G,-4.3,G,64.0,Y,0.0,G,2.0,G,270.0,G
2019-12-31 22:00:00,1021.5,G,1.7,G,-4.1,G,65.0,Y,0.0,G,2.0,G,260.0,G


In [15]:
pv_syst = ef.PVArray(capacity=4,                   
                     efficiency=0.25)

bess_syst = ef.Battery(storage_capacity=8,   
                       min_soc=0.1,               
                       max_charge=0.5,           
                       max_discharge=0.5,     
                       charge_efficiency=1, 
                       discharge_efficiency=1)

building = ef.Building(assets=[pv_syst, bess_syst], name="My building")


file_path = f"Meteorological_data_Karlstad\Data_processed\Meteo_data_2015.csv"
df_meteo = pd.read_csv(file_path)
df_meteo.set_index(df_meteo.columns[0], inplace=True)

#Evaluation of the PV production/Users/sebaheg/Documents/electricity_demand.csv
df_inputs['PVProduction'] = PV_AC_Output(
                                    PV_rated_power=pv_syst.capacity,
                                    start_day=start_day,
                                    end_day=end_day,
                                    GHI=df_meteo['SolarGHI'].values,
                                    clear_sky_condition=False
)

#Create the Dataset
PVBessDataset = ef.Dataset(name='Building 1',
                           energy_system=building,
                           description='PV-BESS system',
                           data=df_inputs
)

FileNotFoundError: [Errno 2] No such file or directory: 'Meteorological_data_Karlstad\\Data_processed\\Meteo_data_2015.csv'